# MOME x MTMT próbafeladat

### Az MTMT egy tudományos és irodalmi publikációkat, továbbá művészeti alkotásokat tartalmazó, nyilvánosan hozzáférhető, json-formátumban lekérhezhető adatbázis. Feladatod ennek az adatbázisnak parciális megemésztése. Elsődlegesen pythonban...pandaszelídítés erősen javallott.
##### Határidő: Az Univerzum hőhalála

### Import

In [62]:
import pandas as pd
import urllib
import json
from tqdm import tqdm

### MTMT API-wrapperek és nested-json helperek

In [63]:
def get_creators(affiliation_code="208"):
    mtmt_link = f"https://m2.mtmt.hu/api/author?cond=affiliations.worksFor;inica;{affiliation_code}&sort=label,asc&size=5000&format=json"
    with urllib.request.urlopen(mtmt_link) as url:
        d = json.loads(url.read().decode())
    df_creators_nested_list = pd.json_normalize(d['content'])
    creators = df_creators_nested_list[["mtid","familyName","givenName","auxName"]].rename(columns={"mtid":"author_mtid","auxName":"FIELD"}).copy()
    return creators

def get_creations(author_code):
    mtmt_link = fr"https://m2.mtmt.hu/api/publication?cond=authorships.author;eq;{author_code}&size=5000&labelLang=hun&cite_type=2&format=json"
    with urllib.request.urlopen(mtmt_link) as url:
        d = json.loads(url.read().decode())
    df_i_nested_list = pd.json_normalize(d['content'])
    df_i_nested_list["author_mtid"] = author_code
    return df_i_nested_list


In [64]:
def flatten_nested_json_df(df):
    try:
        df = df.reset_index()
    except:
        pass

    s = (df.applymap(type) == list).all()
    list_columns = s[s].index.tolist()

    s = (df.applymap(type) == dict).all()
    dict_columns = s[s].index.tolist()
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []

        for col in dict_columns:
            horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horiz_exploded.index = df.index
            df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
            new_columns.extend(horiz_exploded.columns) # inplace

        for col in list_columns:
            df = df.drop(columns=[col]).join(df[col].explode().to_frame())
            new_columns.append(col)
        s = (df[new_columns].applymap(type) == list).all()
        list_columns = s[s].index.tolist()

        s = (df[new_columns].applymap(type) == dict).all()
        dict_columns = s[s].index.tolist()
    return df

### MOME 'alkotók'

In [65]:
MOME_creators = get_creators(affiliation_code="208")
MOME_creators.head()

,author_mtid,familyName,givenName,FIELD
0,10014956,Antalóczy,Tímea,szociológia
1,10025242,Babarczy,Eszter,kultúrakutatás
2,10076778,Bagdi,Sára,"művészettörténet, historiográfia, esztétika, k..."
3,10072202,Bakk,Ágnes Karolina,interaktív narratív design
4,10068336,Balla,Dóra Judit,Tervezőgrafika történet és elmélet


### 1. feladat: Generálj egy saját 'belső id'-t, aminek segítségével a duplikált szerzőket (pl. Bene Tamás, Handú Anett) össze lehet majd gyúrni!

### MTMT publikációk és alkotások

In [66]:
df_related_creations = pd.DataFrame()

print("Accessing MTMT database...") # kb. 5-10 perc az MTMT terheltségétől függően
for author_mtid in tqdm(MOME_creators['author_mtid'].to_list()):
    df_creations = get_creations(author_code=author_mtid)
    df_related_creations = pd.concat([df_creations,df_related_creations], ignore_index=True)

Accessing MTMT database...


100%|██████████| 178/178 [03:56<00:00,  1.33s/it]


### 2. feladat/kérdés: szerinted meggyorsítható ez a lekérdezés, ha automatizálni szeretnénk a folyamatot? Nem kell kidolgoznod, de ha kedved szottyan hozzá, nyugodtan kísérletezhetsz az MTMT api függvényekkel.

### MTMT tevékenység alapadatok

In [67]:
df_creations = df_related_creations[["mtid","author_mtid","creator.mtid","title","subTitle","type.label","subType.docType.label","subType.name","category.label","comment","abstractText","publishedYear","collaboration","label","authorships","subjects","keywords","properties"]]
df_creations.head()

,mtid,author_mtid,creator.mtid,title,subTitle,type.label,subType.docType.label,subType.name,category.label,comment,abstractText,publishedYear,collaboration,label,authorships,subjects,keywords,properties
0,32236397.0,10058347,10071569.0,Péter Vladimir könyve,NaN,Könyv,Könyv,Monográfia,Tudományos,NaN,Péter Vladimír munkásságat bemutató kötet. Fot...,2007.0,NaN,Máté Gábor et al. Péter Vladimir könyve. (2007...,"[{'otype': 'PersonAuthorship', 'mtid': 9762052...",NaN,NaN,NaN
1,31335579.0,10058347,10058346.0,Péter Vladimir könyve,NaN,Könyv,Könyv,Monográfia,Ismeretterjesztő,NaN,NaN,2007.0,NaN,Vargha Balázs et al. Péter Vladimir könyve. (2...,"[{'otype': 'PersonAuthorship', 'mtid': 9178525...",NaN,"[{'otype': 'Keyword', 'mtid': 2017487, 'link':...",NaN
2,33532388.0,10010868,10010868.0,Magyar Modern – Ungarische Kunst in Berlin 191...,NaN,Könyv,Könyv,Katalógus,Tudományos,NaN,NaN,2022.0,NaN,Burmeister Ralf. Magyar Modern – Ungarische Ku...,"[{'otype': 'PersonAuthorship', 'mtid': 1061392...",NaN,NaN,NaN
3,33540172.0,10010868,10040500.0,Magyar Modern: Hungarian Art in Berlin 1910–1933,NaN,Könyv,Könyv,Szakkönyv,Tudományos,NaN,NaN,2022.0,NaN,Ralf Burmeister. Magyar Modern: Hungarian Art ...,"[{'otype': 'PersonAuthorship', 'mtid': 1061783...",NaN,NaN,NaN
4,33532389.0,10010868,10010868.0,"„Ungarn gibt es überall” – Kontakte, Freundsch...",NaN,Könyvrészlet,Könyvrészlet,Szaktanulmány,Tudományos,NaN,NaN,2022.0,NaN,Zwickl András. „Ungarn gibt es überall” – Kont...,"[{'otype': 'PersonAuthorship', 'mtid': 1061393...",NaN,NaN,NaN


### 3. feladat: Dolgozd át a 'comment', 'abstracttext' és 'label' oszlopokat egy közös 'general_description'-re.! Próbálj minél magasabb minőségű szöveget generálni (szerzőnevek, fura azonosítók és évszámok nélkül)!

### 4. feladat: A 'keywords', 'subject' és 'properties' oszlopok további hasznos infókat tartalmaz, próbáld ezeket is kinyerni a flatten_nested_json_df függvénnyel!
### 'Vegyük észre', hogy a 'properties' egy újabb 'tevékenység' kibányászását is lehetővé teszi (általában alkotás-típusú tevékenységeknél). Hívjuk ezt 'kiállításnak'. A helyszín/megjelenés helye (város ill. ország), a megjelenés kezdete/vége, továbbá a megjelenés jellege alapján generálj egy újabb táblát ('df_exhibitions'), majd hozd azt a 'df_creations' táblához hasonló formátumra. A lényeg, hogy legyen egy nagy közös, standardizált 'tevékenység' táblánk, minél több és minél jobb minőségű adattal.

### 5. feladat: Az authors_df tábla a tevékenységekhez köthető szerepeket (is) gyűjti. Egészítsd ki az authors_df táblát is a kiállításokkal!

In [68]:
authors_df = flatten_nested_json_df(df_related_creations[["mtid","authorships"]])[["mtid","authorships.author.mtid","authorships.familyName","authorships.givenName","authorships.type.label","authorships.label"]].drop_duplicates()
authors_df.head()

,mtid,authorships.author.mtid,authorships.familyName,authorships.givenName,authorships.type.label,authorships.label
0,32236397.0,10071569.0,Máté,Gábor,Szerző,"Máté, Gábor [Máté, Gábor (Fotográfia), szerző]"
0,32236397.0,10058346.0,Vargha,Balázs,Szerző,"Vargha, Balázs [Vargha, Balázs (médiaművészet)..."
0,32236397.0,10058347.0,Zsótér,László,Szerző,"Zsótér, László [Zsótér, László (tervezőgrafika..."
1,31335579.0,10058346.0,Vargha,Balázs,Tervező,"Vargha, Balázs [Vargha, Balázs (médiaművészet)..."
1,31335579.0,10058742.0,Péter,Vladimir,Szerző,"Péter, Vladimir [Péter, Vladimir (Fémművesség,..."


#### Várt végeredmény: Legyen legalább 3 standard tábla
- Egy általános 'user' tábla
- a standardizált MTMT tevékenységeket gyűjtő tábla,
- adott tevékenységehez köthető userek, és azok aktuális szerepét tartalmazzó tábla

##### Van még valami, amire szerinted érdemes lenne odafigyelni?

### 6. feladat: Alap adatvizualizációk, az elemzés mélységét és scope-ját te határozhatod meg, de mindenképp legyen folytonos és kategorikus változók ábrázolása. Nem kell szépnek lennie, de legyen informatív.
##### Ha nagyon kell inspiráció; legyen mondjuk: "Mi a MOME, mint egyetem, MTMT-profilja, vagy milyen user profilok vannak a MOME-n?"

### 7. tervezési feladat: Vázolj fel egy 'search app'-ot! A cél hogy, egy sima szövegkereső alapján egyszerre tudj tevékenységeke és személyekre is keresni. Gondolkozz azon is, hogy hogyan lehetne az egymáshoz leghasonlóbb / egymáshoz 'legközelebbi' entitásokat (mind tevékenység, mind személyszinten) megtalálni. Szándékosan ilyen aspecifikus a specifikáció, itt inkább a 'brainstorm' képesség demonstrálása a cél. Milyen megközelítésekben gondolkodnál, milyen package-ekkel villognál?
##### Értelemszerűen nem kell implementálni, csak egy pármondatos vázlat a cél.